In [ ]:
cd <your path>

# Begining

## Load modules

In [ ]:
! pip install pyspark

In [ ]:
from operator import itemgetter
import time
import sys
import getpass
from pathlib import Path
import pandas as pd
import numpy as np

import itertools

## Define the Parameters

In [ ]:
# PATH = '/scratch/work/courses/DSGA1004-2021/MSD'
PATH = './data'

STATS_PATH = Path('./saved_tuned_model_stats')

WHOLE_DATA = True
FRACTION = 1.00
TOP=500
PREC_AT = 500

# Load the data

In [ ]:
if WHOLE_DATA:
    train_pd = pd.read_parquet(PATH+'/cf_train_trans.parquet')
    unique_user_index_val_pd = pd.read_parquet(PATH+'/unique_user_index_val.parquet')
else:
    train_pd = pd.read_parquet(PATH+'/cf_train_trans_{}.parquet'.format(str(FRACTION).replace('.','_')))
    unique_user_index_val_pd = pd.read_parquet(PATH+'/unique_user_index_val.parquet')
    unique_user_index_val_pd = unique_user_index_val_pd.sample(replace=False, frac=max(FRACTION,0.05)) 

val_pd = pd.read_parquet(PATH+'/cf_validation_trans.parquet')
print('Successfully loaded the data')

Successfully loaded the data


In [ ]:
train_pd.rename(columns={'user_index':'user','track_index':'item','count':'rating'}, inplace=True)
train_pd.head(2)

,user_id,rating,track_id,__index_level_0__,user,item
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,TRIQAUQ128F42435AD,0,811701,127138
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1,TRIRLYL128F42539D1,1,811701,127949


In [ ]:
unique_user_index_val_pd = np.array(unique_user_index_val_pd.values.tolist())
unique_user_index_val_list = list(unique_user_index_val_pd.reshape(-1))

In [ ]:
val_pd.rename(columns={'user_index':'user','track_index':'item','count':'rating'}, inplace=True)
val_pd.head(2)

,user_id,rating,track_id,__index_level_0__,user,item
0,0007140a3796e901f3190f12e9de6d7548d4ac4a,1,TRUFCYO128F422B898,0,109,300356
1,0007140a3796e901f3190f12e9de6d7548d4ac4a,2,TROBZPR128F14808FF,1,109,209116


# ALS Model lenskit

In [ ]:
! pip install lenskit

     |████████████████████████████████| 92kB 4.5MB/s 
     |████████████████████████████████| 133kB 18.4MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219250 sha256=e6e8018d5fbe93f4b402366e734c9657b59b4df3290166399d727afb7161b80f
  Stored in directory: /root/.cache/pip/wheels/a6/90/95/f889ca4aa8b0e0c7f21c8470b6f5d6032f0390a3a141a9a3bd
Successfully built pickle5


In [ ]:
from lenskit.algorithms import als as ALS_ext
from lenskit.algorithms import Recommender
from lenskit import batch, topn, util

In [ ]:
ranks = [i for i in range(10,40,10)]
# ranks = [i for i in range(100,500,30)]
regParams = [1] #[10**i for i in range(-2,1)   
alphas = [1]
maxIter = [5]

params = [[a,b,c,d] for a,b,c,d in itertools.product(ranks, regParams, alphas, maxIter)]
print('length of params',len(params))

precisionAt_k_highest = 0
stats_file = open(STATS_PATH/'stats_lenskit_{}.txt'.format(str(FRACTION).replace('.','_')), 'a', buffering=1)
print(' '.join(sys.argv))
print(' '.join(sys.argv), file=stats_file)
for rk,reg,alp,it in itertools.product(ranks, regParams, alphas, maxIter):
    since = time.time()
    als_ext = ALS_ext.ImplicitMF(features=rk, iterations=it, reg=reg, weight=alp)
    als_ext = Recommender.adapt(als_ext)

    model_ext = als_ext.fit(train_pd)
    time_to_fit = time.time() - since

    pred_recs_tracks_val_ext = batch.recommend(model_ext, unique_user_index_val_list, TOP)
    time_to_recommend = time.time() - since

    rla = topn.RecListAnalysis()
    rla.add_metric(topn.precision)
    results = rla.compute(pred_recs_tracks_val_ext, val_pd)
    precisionAt_k = results.precision.mean()
    time_to_eval = time.time() - since
    
    print("With Rank:{}, Reg:{}, Alpha:{}, Maxiter:{}, Metric: {}, time TTL: {}, fit: {}, rec: {}, evl: {}".format(
        rk,reg,alp,it, 
        precisionAt_k, 
        time_to_eval, time_to_fit, time_to_recommend-time_to_fit, time_to_eval-time_to_recommend 
    ), file=stats_file)

    print("With Rank:{}, Reg:{}, Alpha:{}, Maxiter:{}, Metric: {}, time TTL: {}, fit: {}, rec: {}, evl: {}".format(
        rk,reg,alp,it, 
        precisionAt_k, 
        time_to_eval, time_to_fit, time_to_recommend-time_to_fit, time_to_eval-time_to_recommend 
    ))

length of params 3
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-e5454f5f-1a37-42a9-98b2-37175efd6ed1.json
With Rank:10, Reg:1, Alpha:1, Maxiter:5, Metric: 0.007356200000000163, time TTL: 637.7977559566498, fit: 93.50110816955566, rec: 456.71342635154724, evl: 87.58322143554688
With Rank:20, Reg:1, Alpha:1, Maxiter:5, Metric: 0.008199000000000133, time TTL: 698.7235927581787, fit: 127.64269423484802, rec: 485.28489089012146, evl: 85.79600763320923
With Rank:30, Reg:1, Alpha:1, Maxiter:5, Metric: 0.008628600000000139, time TTL: 785.9004993438721, fit: 165.6219139099121, rec: 531.1211321353912, evl: 89.15745329856873


In [ ]:
ranks = [i for i in range(100,320,30)]
# ranks = [i for i in range(100,500,30)]
regParams = [1] #[10**i for i in range(-2,1)   
alphas = [1]
maxIter = [5]

params = [[a,b,c,d] for a,b,c,d in itertools.product(ranks, regParams, alphas, maxIter)]
print('length of params',len(params))

precisionAt_k_highest = 0
stats_file = open(STATS_PATH/'stats_lenskit_{}.txt'.format(str(FRACTION).replace('.','_')), 'a', buffering=1)
print(' '.join(sys.argv))
print(' '.join(sys.argv), file=stats_file)
for rk,reg,alp,it in itertools.product(ranks, regParams, alphas, maxIter):
    since = time.time()
    als_ext = ALS_ext.ImplicitMF(features=rk, iterations=it, reg=reg, weight=alp)
    als_ext = Recommender.adapt(als_ext)

    model_ext = als_ext.fit(train_pd)
    time_to_fit = time.time() - since

    pred_recs_tracks_val_ext = batch.recommend(model_ext, unique_user_index_val_list, TOP)
    time_to_recommend = time.time() - since

    rla = topn.RecListAnalysis()
    rla.add_metric(topn.precision)
    results = rla.compute(pred_recs_tracks_val_ext, val_pd)
    precisionAt_k = results.precision.mean()
    time_to_eval = time.time() - since
    
    print("With Rank:{}, Reg:{}, Alpha:{}, Maxiter:{}, Metric: {}, time TTL: {}, fit: {}, rec: {}, evl: {}".format(
        rk,reg,alp,it, 
        precisionAt_k, 
        time_to_eval, time_to_fit, time_to_recommend-time_to_fit, time_to_eval-time_to_recommend 
    ), file=stats_file)

    print("With Rank:{}, Reg:{}, Alpha:{}, Maxiter:{}, Metric: {}, time TTL: {}, fit: {}, rec: {}, evl: {}".format(
        rk,reg,alp,it, 
        precisionAt_k, 
        time_to_eval, time_to_fit, time_to_recommend-time_to_fit, time_to_eval-time_to_recommend 
    ))

length of params 8
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-e5454f5f-1a37-42a9-98b2-37175efd6ed1.json
With Rank:100, Reg:1, Alpha:1, Maxiter:5, Metric: 0.009993600000000182, time TTL: 1365.3922743797302, fit: 503.0088527202606, rec: 773.9596619606018, evl: 88.4237596988678
